In [1]:
import re
import os
from docx import Document

: 

In [ ]:

import win32com.client
import fitz
import spacy
from langdetect import detect
import logging
from PIL import Image
import pytesseract
from statistics import mean
from collections import defaultdict

In [ ]:
logging.basicConfig(level=logging.INFO)
nlp = spacy.load("xx_ent_wiki_sm")

def sentence_end(text):
    """
    Kiểm tra xem chuỗi văn bản có kết thúc bằng dấu câu hợp lệ hoặc cặp ngoặc hợp lệ hay không.
    """
    brackets = ["()", "''", '""', "[]", "{}", "«»", "“”", "‘’"]
    valid_brackets = any(text.startswith(pair[0]) and text.endswith(pair[1]) for pair in brackets)
    valid_end = text.endswith(('.', '!', '?', ':', ';'))
    return valid_end or valid_brackets

def markers(text):
    """
    Kiểm tra xem chuỗi văn bản có khớp với cấu trúc tiêu đề hay không.
    """
    patterns = [
        r"^[A-ZÀ-Ỹa-zà-ỹ]+\s+[0-9IVXLCDM]+[:.\s].*",  # Chương 1, Điều 1, Section I
        r"^[0-9]+[\.\)]\s*.*",  # 1., 1)
        r"^[IVXLCDM]+[\.\)]\s*.*",  # I., II)
        r"^[a-zA-Z][\.\)]\s*.*",  # a., b)
        r"^[0-9]+\.[0-9]+[\.\)]\s*.*",  # 1.1, 1.2
        r"^[A-ZÀ-Ỹ][A-ZÀ-Ỹ\s]{2,}$",  # QUY ĐỊNH CHUNG
    ]
    return any(re.match(pattern, text, re.IGNORECASE) for pattern in patterns)

def is_title_by_nlp(text, prev_text):
    """
    Sử dụng NLP để xác định dòng có phải tiêu đề hay không.
    """
    doc = nlp(text)
    if len(doc) > 10 or any(token.pos_ == "VERB" for token in doc):
        return False
    return len(doc) < 10 and any(token.pos_ in ["NOUN", "PROPN"] for token in doc)

def unclosed(text):
    """
    Kiểm tra xem chuỗi văn bản có chứa ngoặc chưa được đóng hay không.
    """
    stack = []
    brackets = {"(": ")", "[": "]", "{": "}", '"': '"', "'": "'", "«": "»", "“": "”", "‘": "’"}
    for char in text:
        if char in brackets.keys():
            stack.append(char)
        elif char in brackets.values():
            if stack and brackets[stack[-1]] == char:
                stack.pop()
            else:
                return False
    return bool(stack)

def merge_text(para, new_para, is_para_title, is_new_para_title, prev_text):
    """
    Quyết định xem có nên gộp hai đoạn văn bản thành một hay không.
    """
    if is_new_para_title or is_para_title:
        return False
    return (
        not markers(new_para) and
        (not new_para[0].isupper() or not sentence_end(para)) or
        unclosed(para)
    )

def extract_text_from_image_page(page):
    """
    Trích xuất văn bản từ trang PDF quét bằng OCR.
    """
    pix = page.get_pixmap()
    img = Image.frombytes("RGB", [pix.width, pix.height], pix.rgb)
    return pytesseract.image_to_string(img, lang="vie+eng")

def learn_title_hierarchy(doc):
    """
    Duyệt văn bản để xác định cấp độ tiêu đề dựa trên đoạn có nhiều cấp nhất.
    """
    segments = []
    for page in doc:
        blocks = sorted(page.get_text("blocks"), key=lambda b: (b[1], b[0]))
        segment = []
        prev_text = ""
        prev_block = None
        for block in blocks:
            font_info = block[6] if len(block) > 6 else {}
            x0 = block[0]
            spacing = (block[1] - prev_block[3] if prev_block else 0)
            is_isolated = spacing > 10
            for line in block[4].split("\n"):
                cleaned_text = " ".join(line.strip().split())
                if cleaned_text:
                    doc = nlp(cleaned_text)
                    is_content = len(doc) > 10 or any(token.pos_ == "VERB" for token in doc)
                    if is_content or not is_isolated:
                        continue
                    if markers(cleaned_text) or is_title_by_nlp(cleaned_text, prev_text):
                        segment.append({
                            "text": cleaned_text,
                            "indent": x0,
                            "font_size": font_info.get("size", 10),
                            "bold": font_info.get("weight", "").lower() == "bold",
                            "italic": font_info.get("flags", 0) & 2 != 0,
                            "alignment": "center" if abs(block[0] - block[2]) < abs(block[1] - block[3]) else "left"
                        })
                prev_text = cleaned_text
                prev_block = block
        if segment:
            segments.append(segment)
    
    # Tìm đoạn có nhiều cấp nhất
    max_levels = 0
    selected_segment = None
    for segment in segments:
        indents = set(item["indent"] for item in segment)
        if len(indents) > max_levels:
            max_levels = len(indents)
            selected_segment = segment
    
    if not selected_segment:
        return {}
    
    # Xác định cấp độ dựa trên lùi đầu dòng và font size
    title_formats = {}
    current_level = 1
    seen_patterns = set()
    sorted_segment = sorted(selected_segment, key=lambda x: (x["indent"], -x["font_size"]))
    for item in sorted_segment:
        pattern = re.match(r"^[A-ZÀ-Ỹa-zà-ỹ]+\s+[0-9IVXLCDM]+|[0-9a-zA-ZIVXLCDM]+[\.\)]|[A-ZÀ-Ỹ\s]{2,}", item["text"], re.IGNORECASE)
        if pattern:
            pattern_key = pattern.group(0)
            if pattern_key not in seen_patterns:
                seen_patterns.add(pattern_key)
                title_formats[f"level_{current_level}"] = {
                    "pattern": pattern_key,
                    "font_size": item["font_size"],
                    "bold": item["bold"],
                    "italic": item["italic"],
                    "alignment": item["alignment"],
                    "indent": item["indent"],
                    "count": 1
                }
                current_level += 1
    
    # Gộp định dạng từ các đoạn khác
    for segment in segments:
        for item in segment:
            for level, fmt in title_formats.items():
                if re.match(rf"^{fmt['pattern']}", item["text"], re.IGNORECASE):
                    fmt["font_size"] = (fmt["font_size"] * fmt["count"] + item["font_size"]) / (fmt["count"] + 1)
                    fmt["count"] += 1
    
    return title_formats

def is_title(block, text, prev_block, prev_text, title_formats):
    """
    Xác định dòng có phải tiêu đề dựa trên định dạng và cấp độ đã học.
    """
    if not (markers(text) or is_title_by_nlp(text, prev_text)):
        return False
    
    font_info = block[6] if len(block) > 6 else {}
    font_size = font_info.get("size", 10)
    is_bold = font_info.get("weight", "").lower() == "bold"
    is_italic = font_info.get("flags", 0) & 2 != 0
    alignment = "center" if abs(block[0] - block[2]) < abs(block[1] - block[3]) else "left"
    indent = block[0]
    spacing = (block[1] - prev_block[3] if prev_block else 0)
    is_isolated = spacing > 10
    
    if not is_isolated:
        return False
    
    for level, fmt in title_formats.items():
        if re.match(rf"^{fmt['pattern']}", text, re.IGNORECASE):
            font_size_match = abs(font_size - fmt["font_size"]) <= 1
            bold_match = is_bold == fmt["bold"]
            italic_match = is_italic == fmt["italic"]
            alignment_match = alignment == fmt["alignment"]
            indent_match = abs(indent - fmt["indent"]) <= 5
            if sum([font_size_match, bold_match, italic_match, alignment_match, indent_match]) >= 5:
                return True
    return False

def validate_segments(text_data):
    """
    Kiểm tra và gộp các đoạn ngắn bất thường.
    """
    validated_data = []
    for i, segment in enumerate(text_data):
        if len(segment["text"].split()) < 5 and not segment["metadata"]["is_title"]:
            if i > 0 and not text_data[i-1]["metadata"]["is_title"]:
                validated_data[-1]["text"] += " " + segment["text"]
                continue
        validated_data.append(segment)
    return validated_data

def extracted(path):
    """
    Trích xuất văn bản từ file (.docx, .doc, .pdf) và tổ chức thành các đoạn.
    """
    try:
        file_ext = os.path.splitext(path)[1].lower()
        text_data = []

        if file_ext == ".docx":
            doc = Document(path)
            paragraph = ""
            is_para_title = False
            prev_text = ""
            for para in doc.paragraphs:
                for line in para.text.split("\n"):
                    cleaned_text = ' '.join(line.strip().split())
                    if cleaned_text:
                        is_new_para_title = markers(cleaned_text) or is_title_by_nlp(cleaned_text, prev_text)
                        if paragraph and merge_text(paragraph, cleaned_text, is_para_title, is_new_para_title, prev_text):
                            paragraph += " " + cleaned_text
                        else:
                            if paragraph:
                                text_data.append({"text": paragraph, "metadata": {"is_title": is_para_title}})
                            paragraph = cleaned_text
                            is_para_title = is_new_para_title
                        prev_text = cleaned_text
            if paragraph:
                text_data.append({"text": paragraph, "metadata": {"is_title": is_para_title}})
            return validate_segments(text_data)

        elif file_ext == ".doc":
            word = win32com.client.Dispatch("Word.Application")
            word.Visible = False
            doc = word.Documents.Open(os.path.abspath(path))
            text = doc.Content.Text
            doc.Close()
            word.Quit()

            paragraph = ""
            is_para_title = False
            prev_text = ""
            for line in text.split("\n"):
                cleaned_text = ' '.join(line.strip().split())
                if cleaned_text:
                    is_new_para_title = markers(cleaned_text) or is_title_by_nlp(cleaned_text, prev_text)
                    if paragraph and merge_text(paragraph, cleaned_text, is_para_title, is_new_para_title, prev_text):
                        paragraph += " " + cleaned_text
                    else:
                        if paragraph:
                            text_data.append({"text": paragraph, "metadata": {"is_title": is_para_title}})
                        paragraph = cleaned_text
                        is_para_title = is_new_para_title
                    prev_text = cleaned_text
            if paragraph:
                text_data.append({"text": paragraph, "metadata": {"is_title": is_para_title}})
            return validate_segments(text_data)

        elif file_ext == ".pdf":
            doc = fitz.open(path)
            sample_text = " ".join(page.get_text("text") for page in doc[:2] if page.get_text("text").strip())
            language = detect(sample_text) if sample_text else "unknown"
            
            # Học cấp độ và định dạng tiêu đề
            title_formats = learn_title_hierarchy(doc)
            logging.info(f"Learned title formats: {title_formats}")
            
            paragraph = ""
            is_para_title = False
            prev_text = ""
            prev_block = None
            for page in doc:
                if not page.get_text("text").strip():
                    text = extract_text_from_image_page(page)
                    blocks = [(0, 0, 0, 0, text, 0, {})]
                else:
                    blocks = sorted(page.get_text("blocks"), key=lambda b: (b[1], b[0]))
                for i, block in enumerate(blocks):
                    font_info = block[6] if len(block) > 6 else {}
                    next_block = blocks[i+1] if i+1 < len(blocks) else None
                    metadata = {
                        "page": page.number,
                        "font_size": font_info.get("size", 10),
                        "is_bold": font_info.get("weight", "").lower() == "bold",
                        "x0": block[0],
                        "y0": block[1],
                        "language": language
                    }
                    for line in block[4].split("\n"):
                        cleaned_text = " ".join(line.strip().split())
                        if cleaned_text:
                            is_new_para_title = is_title(block, cleaned_text, prev_block, prev_text, title_formats)
                            metadata["is_title"] = is_new_para_title
                            if paragraph and merge_text(paragraph, cleaned_text, is_para_title, is_new_para_title, prev_text):
                                paragraph += " " + cleaned_text
                            else:
                                if paragraph:
                                    text_data.append({"text": paragraph, "metadata": metadata})
                                paragraph = cleaned_text
                                is_para_title = is_new_para_title
                            prev_text = cleaned_text
                    prev_block = block
            if paragraph:
                text_data.append({"text": paragraph, "metadata": metadata})
            return validate_segments(text_data)

        return text_data

    except Exception as e:
        logging.error(f"Error processing {path}: {e}")
        return []



In [ ]:
if __name__ == "__main__":
    chunks = extracted("../Doc/HNMU_Regulations.pdf")
    import json
    with open("output.json", "w", encoding="utf-8") as f:
        json.dump(chunks, f, ensure_ascii=False, indent=2)